# SQL 的五十道練習

> 包含子查詢的結構

[數據交點](https://www.datainpoint.com/) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
%LOAD ../databases/imdb.db

In [2]:
ATTACH "../databases/nba.db" AS nba;

In [3]:
ATTACH "../databases/twElection2020.db" AS twElection2020;

In [4]:
ATTACH "../databases/covid19.db" AS covid19;

## 什麼是子查詢

## 在「函數」的章節中，我們看過函數中包含著其他函數的使用方法

前一個函數的輸出，成為下一個函數的輸入。

In [5]:
-- 函數中包含著其他函數
SELECT ROUND(AVG(rating)) AS avg_rating 
  FROM top_rated_movies;

avg_rating
8.0


## 在一段 SQL 中包含著另外一段 SQL 的使用方法，被稱為子查詢（Subquery）。

## 在什麼樣的情境下我們會想使用子查詢呢？

- 查詢的篩選條件是必須要先做一個查詢才能得知。
- 查詢的計算內容必須要先做一個查詢才能得知。

## 舉例來說，我們想知道 `top_rated_movies` IMDb 最高評價的 250 部電影中最高評價的電影是哪一部？

- 先查詢最高評價的數值為多少。
- 再以前一個查詢結果作為篩選條件。

In [6]:
-- 先查詢最高評價的數值為多少。
SELECT MAX(rating) AS max_rating
  FROM top_rated_movies;

max_rating
9.3


In [7]:
-- 再以前一個查詢結果作為篩選條件。
SELECT title AS best_rated_title
  FROM top_rated_movies
 WHERE rating = 9.3;

best_rated_title
The Shawshank Redemption


In [8]:
-- 在查詢中包含前一個查詢結果的 SQL
SELECT title AS best_rated_title
  FROM top_rated_movies
 WHERE rating = (SELECT MAX(rating) AS max_rating
                   FROM top_rated_movies);

best_rated_title
The Shawshank Redemption


## 舉例來說，我們想知道 `top_rated_movies` IMDb 最高評價的 250 部電影中，在 2000 年之後上映的電影佔比為多少？

- 先查詢在 2000 年之後上映的電影有幾部。
- 再以前一個查詢結果作為計算內容。

In [9]:
-- 先查詢在 2000 年之後上映的電影有幾部。
SELECT COUNT(*) AS n_movies
  FROM top_rated_movies
 WHERE release_year >= 2000;

n_movies
99


In [10]:
-- 再以前一個查詢結果作為計算內容。
SELECT 99 / CAST(COUNT(*) AS REAL) AS millennium_percentage
  FROM top_rated_movies;

millennium_percentage
0.396


In [11]:
-- 在查詢中包含前一個查詢結果的 SQL
SELECT (SELECT COUNT(*) AS n_movies
          FROM top_rated_movies
         WHERE release_year >= 2000) / CAST(COUNT(*) AS REAL) AS millennium_percentage
  FROM top_rated_movies;

millennium_percentage
0.396


## 常見的子查詢可以分為三類：

1. 將一段 SQL 查詢結果作為資料表。
2. 將一段 SQL 查詢結果作為條件。 
3. 將一段 SQL 查詢結果作為衍生計算欄位。

## 將一段 SQL 查詢結果作為資料表

## 在 `FROM` 敘述後接另外一段 SQL 查詢

```sql
SELECT column_name
  FROM (SELECT column_name FROM table_name) AS alias;
```

In [12]:
SELECT *
  FROM (SELECT release_year,
               title,
               rating
          FROM top_rated_movies
         WHERE release_year = 1994) AS top_1994_movies;

release_year,title,rating
1994,The Shawshank Redemption,9.3
1994,Pulp Fiction,8.9
1994,Forrest Gump,8.8
1994,Léon: The Professional,8.5
1994,The Lion King,8.5
1994,Three Colors: Red,8.1


## 將一段 SQL 查詢結果作為條件

## 在 `WHERE` 敘述後接另外一段 SQL 查詢

```sql
SELECT column_name
  FROM table_name
 WHERE condition (SELECT column_name FROM table_name);
```

In [13]:
SELECT *
  FROM top_rated_movies
 WHERE release_year IN (SELECT MIN(release_year)
                         FROM top_rated_movies);

id,title,release_year,rating,director,runtime
101,The Kid,1921,8.3,Charles Chaplin,68


## 在 `HAVING` 敘述後接另外一段 SQL 查詢

```sql
SELECT column_name
  FROM table_name
 GROUP BY column_name
HAVING condition (SELECT column_name FROM table_name);
```

In [14]:
SELECT release_year,
       ROUND(AVG(rating), 1) AS avg_rating 
  FROM top_rated_movies
 GROUP BY release_year
HAVING avg_rating >= (SELECT MAX(rating) - 0.8 FROM top_rated_movies);

release_year,avg_rating
1936,8.5
1946,8.6
1972,9.2
1974,8.6
1977,8.6
1990,8.7
1991,8.6
1994,8.7
1999,8.5
2002,8.5


## 將一段 SQL 查詢結果作為衍生計算欄位

## 在 `SELECT` 敘述後接另外一段 SQL 查詢

```sql
SELECT ... (SELECT column_name FROM table_name)
  FROM table_name;
```

In [15]:
SELECT number,
       candidate,
       ROUND(CAST(SUM(votes) AS REAL) / (SELECT SUM(votes) FROM presidential), 2) AS votes_percentage
  FROM presidential
 GROUP BY number;

number,candidate,votes_percentage
1,宋楚瑜/余湘,0.04
2,韓國瑜/張善政,0.39
3,蔡英文/賴清德,0.57


## 重點統整

- 在一段 SQL 中包含著另外一段 SQL 的使用方法，被稱為子查詢（Subquery）。
- 子查詢常見的應用情境
    - 查詢的篩選條件是必須要先做一個查詢才能得知。
    - 查詢的計算內容必須要先做一個查詢才能得知。
- 常見的子查詢可以分為三類：
    - 將一段 SQL 查詢結果作為資料表。
    - 將一段 SQL 查詢結果作為條件。
    - 將一段 SQL 查詢結果作為衍生計算欄位。

## 目前我們會的 SQL

```sql
-- 搭配子查詢
SELECT column_name
  FROM table_name
 WHERE condition
 GROUP BY column_name
HAVING condition 
 ORDER BY column_name
 LIMIT m;
```